# Final code for R-peak distribution

In [28]:
import os 
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance


In [114]:
def rpeak_distribution(rpeakList, fxvalList, ba_set_time, plot_yesno, plot_range, ecl_dist_yesno, ecl_dist_plot_yesno):
    
    # ba_set_time: set before, after time based on rpeak
    # plot_yesno: if you want to make a plot, write "yes" or not "no"
    # ecl_dist_yesno: if you want to calculate euclidean distance, write "yes" or not  "no"
    # ecl_dist_yesno: if you want to plot the distribution of euclidean distance, write "yes" or not "no"
    
    set_path = r'C:\Users\MI2RL-KHJ\workspace_signal\mit\cutoff_plot'

    plt.ioff()
   
    for i in range(len(rpeakList)):
            
        # Designate PtId, data type, fxvalId
        PtId = rpeakList[i].split('.csv')[0].split('_')[-1] 
        print('Processing >>>>> ', PtId)
        dtype = rpeakList[i].split('.csv')[0].split('_')[-2]

        fxvalId = rpeakList[i].split('.csv')[0].split('_')[-1]

        # make directgory
        if not os.path.exists(os.path.join(set_path, dtype, 'ba'+str(ba_set_time))):
            os.mkdir(os.path.join(set_path, dtype, 'ba'+str(ba_set_time)))
        else:
            pass

        
        # Check id, if they are same, load each file
        if PtId == fxvalId:
            # Load correct rpeaks data
            rpeaks = pd.read_csv(rpeakList[i], header=None)    

            # Load filtered x-value data
            fxvals = pd.read_csv(fxvalList[i], header=None)
            fxvals = fxvals[0].tolist()         
        
        rs = [] 
        intervals = []
        for p in range(len(rpeaks)):
            
            rpeak = rpeaks.loc[p].values
            brpeak = rpeak - ba_set_time  # 10, 15, 20, 95 points before rpeak 
            if brpeak > 0:
                arpeak = rpeak + ba_set_time  # 10, 15, 20, 95 points after rpeak 
                try: 
                    r = fxvals[int(rpeak)]
                    rs.append(r)
                    interval = fxvals[int(brpeak):int(arpeak)]
                    if len(interval) == ba_set_time*2:
                        intervals.append(interval)
                except:
                    pass
        
        # plotting  _ex) 100 r-peaks with before/after 'ba_set_time(10, 15, 20, 95 point)' sample points
        if plot_yesno == 'yes' :
            
            # set range of rpeak samples
            target_range = range(plot_range, plot_range+100)            
            plt.figure(figsize=(30,25))
            n = 1 
            for tr in target_range:
                ax = plt.subplot(10,10, n)
                plt.subplots_adjust(hspace = 0.4)
                plt.title('<{}-{}> R-peak -+{}: {}'.format(PtId, dtype, ba_set_time, tr), fontsize=10)
                ax.plot(intervals[tr])
                ax.scatter(len(intervals[tr])/2,rs[tr], c='r')
                n+=1                
            plt.savefig(r'C:\Users\MI2RL-KHJ\workspace_signal\mit\cutoff_plot\{}\ba{}\R-peak_{}_{}_ba{}.png'.format(dtype, ba_set_time, PtId, dtype, ba_set_time))
            plt.close()
        else:
            pass
        
        
        dist_base = np.median(intervals)      
        # Get eucliean distance             
        if ecl_dist_yesno == 'yes':
            dists =[]
 #            print('length of intervals:', len(intervals), 'length of rs:', len(rs))
            
            for interval in range(len(intervals)):
                if len(intervals[interval]) > 0:
#                     print(len(intervals[interval]), 'dist_base: ', dist_base)
                    dist = distance.euclidean(intervals[interval], dist_base)
#                     print('dist:', dist)
                    dists.append(dist)
                else:
                    pass
            
#             print('<{}-{}: R-peak -+{}> Euclidean distance summary: mean {}, std: {}'.format(PtId, dtype, ba_set_time, np.mean(dists), np.std(dists)))
            with open (r'C:\Users\MI2RL-KHJ\workspace_signal\mit\cutoff_plot\ecl_{}_{}_summary.csv'.format(dtype, ba_set_time), 'a') as f:
                f.write('{}, {}, {}, {}, {}, \n'.format(PtId, dtype, ba_set_time, np.mean(dists), np.std(dists)))
            
            
            if ecl_dist_plot_yesno == 'yes':

                # Plot Euclidean distance distribution
                plt.figure(figsize=(12,10))
                density, bins, _ = plt.hist(dists, density=True, alpha=0.3)
                count, _ = np.histogram(dists, bins)
                plt.title('<{}-{}> Euclidean distance distribution: R-peak -+{}'.format(PtId, dtype, ba_set_time), fontsize=14)
                plt.grid(True, axis='y', color='gray', alpha=0.5, linestyle='--')
                xmin, xmax, ymin, ymax = plt.axis()

                tick = ((bins[-1] - bins[0])/len(bins))/2
                for x, y, num in zip(bins, density, count):
                    if num != 0:
                        plt.text(x+tick, y, num, horizontalalignment='center', verticalalignment='bottom')   

                total = np.sum(count)
                loose = total*0.9     # adjust %
                strict = total*0.8
                dists = sorted(dists)

                loose_text = 'Euclidean distance: {} for 90% R-peak -+{} > number: {},    [total_rpeaks: {}]'.format(np.round(dists[int(loose)], 3), ba_set_time, int(loose), total)
                strict_text = 'Euclidean distance: {} for 80% R-peak -+{} > number: {}'.format(np.round(dists[int(strict)], 3), ba_set_time, int(strict))

                plt.axvline(x=dists[int(loose)], color='b', linestyle='--', label='Loose', ymax=0.9)
                plt.axvline(x=dists[int(strict)], color='r', linestyle='--', label='Strict', ymax=0.9)
                plt.legend(loc='upper right', fontsize=13)
                plt.text(dists[int(loose)], ymax*0.9, int(loose), fontsize=9, color='b')
                plt.text(dists[int(strict)], ymax*0.9, int(strict), fontsize=9,  color='r')
                plt.text(xmin+xmax/len(bins)*0.05, ymin-np.max(density)*0.1, loose_text, fontsize=14)
                plt.text(xmin+xmax/len(bins)*0.05, ymin-np.max(density)*0.15, strict_text, fontsize=14)
                plt.savefig(r'C:\Users\MI2RL-KHJ\workspace_signal\mit\cutoff_plot\{}\ba{}\R-peak_ecl_{}_{}_ba{}.png'.format(dtype, ba_set_time, PtId, dtype, ba_set_time))             
                plt.close()
            else:
                pass
            
        else:
            pass
#         break

    with open (r'C:\Users\MI2RL-KHJ\workspace_signal\mit\cutoff_plot\ecl_{}_{}_summary.csv'.format(dtype, ba_set_time), 'a') as f:
        f.write('PtId, dtype, before/after, mean, std, \n')
            
    print('{}-before/after{} => Finished'.format(dtype, ba_set_time))                                                                                        
    


In [116]:
PATH_IN = r'C:\Users\MI2RL-KHJ\workspace_signal\mit\processed_norm'
os.chdir(PATH_IN)
rpeakList = [i for i in os.listdir(PATH_IN) if i.startswith('correct')]
fxvalList = [i for i in os.listdir(PATH_IN) if i.startswith('filtered')]
rpeak_distribution(rpeakList, fxvalList, 95, 'no', 100, 'yes', 'no')
rpeak_distribution(rpeakList, fxvalList, 15, 'no', 100, 'yes', 'no')
rpeak_distribution(rpeakList, fxvalList, 10, 'no', 100, 'yes', 'no')

Processing >>>>>  00
Processing >>>>>  01
Processing >>>>>  02
Processing >>>>>  03
Processing >>>>>  04
Processing >>>>>  05
Processing >>>>>  06
Processing >>>>>  07
Processing >>>>>  08
Processing >>>>>  09
Processing >>>>>  10
Processing >>>>>  11
Processing >>>>>  12
Processing >>>>>  13
Processing >>>>>  14
Processing >>>>>  15
Processing >>>>>  16
Processing >>>>>  17
norm-before/after15 => Finished
Processing >>>>>  00
Processing >>>>>  01
Processing >>>>>  02
Processing >>>>>  03
Processing >>>>>  04
Processing >>>>>  05
Processing >>>>>  06
Processing >>>>>  07
Processing >>>>>  08
Processing >>>>>  09
Processing >>>>>  10
Processing >>>>>  11
Processing >>>>>  12
Processing >>>>>  13
Processing >>>>>  14
Processing >>>>>  15
Processing >>>>>  16
Processing >>>>>  17
norm-before/after10 => Finished


In [117]:
PATH_IN = r'C:\Users\MI2RL-KHJ\workspace_signal\mit\processed_arr'
os.chdir(PATH_IN)
rpeakList = [i for i in os.listdir(PATH_IN) if i.startswith('correct')]
fxvalList = [i for i in os.listdir(PATH_IN) if i.startswith('filtered')]
rpeak_distribution(rpeakList, fxvalList, 95, 'no', 100, 'yes', 'no')
rpeak_distribution(rpeakList, fxvalList, 15, 'no', 100, 'yes', 'no')
rpeak_distribution(rpeakList, fxvalList, 10, 'no', 100, 'yes', 'no')

Processing >>>>>  00
Processing >>>>>  01
Processing >>>>>  02
Processing >>>>>  03
Processing >>>>>  04
Processing >>>>>  05
Processing >>>>>  06
Processing >>>>>  07
Processing >>>>>  08
Processing >>>>>  09
Processing >>>>>  10
Processing >>>>>  11
Processing >>>>>  12
Processing >>>>>  13
Processing >>>>>  14
Processing >>>>>  15
Processing >>>>>  16
Processing >>>>>  17
Processing >>>>>  18
Processing >>>>>  19
Processing >>>>>  20
Processing >>>>>  21
Processing >>>>>  22
Processing >>>>>  23
Processing >>>>>  24
Processing >>>>>  25
Processing >>>>>  26
Processing >>>>>  27
Processing >>>>>  28
Processing >>>>>  29
Processing >>>>>  30
Processing >>>>>  31
Processing >>>>>  32
Processing >>>>>  33
Processing >>>>>  34
Processing >>>>>  35
Processing >>>>>  36
Processing >>>>>  37
Processing >>>>>  38
Processing >>>>>  39
Processing >>>>>  40
Processing >>>>>  41
Processing >>>>>  42
Processing >>>>>  43
Processing >>>>>  44
Processing >>>>>  45
Processing >>>>>  46
Processing >>

In [118]:
PATH_IN = r'C:\Users\MI2RL-KHJ\workspace_signal\mit\processed_arrx'
os.chdir(PATH_IN)
rpeakList = [i for i in os.listdir(PATH_IN) if i.startswith('correct')]
fxvalList = [i for i in os.listdir(PATH_IN) if i.startswith('filtered')]
rpeak_distribution(rpeakList, fxvalList, 95, 'no', 100, 'yes', 'no')
rpeak_distribution(rpeakList, fxvalList, 15, 'no', 100, 'yes', 'no')
rpeak_distribution(rpeakList, fxvalList, 10, 'no', 100, 'yes', 'no')

Processing >>>>>  08
Processing >>>>>  09
Processing >>>>>  10
Processing >>>>>  11
Processing >>>>>  12
Processing >>>>>  13
Processing >>>>>  14
Processing >>>>>  15
Processing >>>>>  16
Processing >>>>>  19
Processing >>>>>  20
Processing >>>>>  21
Processing >>>>>  22
Processing >>>>>  38
Processing >>>>>  39
Processing >>>>>  40
Processing >>>>>  41
Processing >>>>>  42
Processing >>>>>  43
Processing >>>>>  44
Processing >>>>>  45
Processing >>>>>  46
Processing >>>>>  47
arrx-before/after95 => Finished
Processing >>>>>  08
Processing >>>>>  09
Processing >>>>>  10
Processing >>>>>  11
Processing >>>>>  12
Processing >>>>>  13
Processing >>>>>  14
Processing >>>>>  15
Processing >>>>>  16
Processing >>>>>  19
Processing >>>>>  20
Processing >>>>>  21
Processing >>>>>  22
Processing >>>>>  38
Processing >>>>>  39
Processing >>>>>  40
Processing >>>>>  41
Processing >>>>>  42
Processing >>>>>  43
Processing >>>>>  44
Processing >>>>>  45
Processing >>>>>  46
Processing >>>>>  47
ar